# Example Workflow - Open Space Benefit

## Data
* US Voter districts: https://www.arcgis.com/home/item.html?id=42fb78fdedf248e89bfa5185651b32e9
* NC Voting districts: https://www.nconemap.gov/datasets/771ae6473a1c4ba3bc768cc2c4b10015_0/explore
* NC Voting Data - https://dl.ncsbe.gov/index.html?prefix=ENRS/2024_11_05/
* 

In [2]:
#Packages
import pandas as pd

from arcgis import GIS
from arcgis.features import FeatureLayer, GeoAccessor

gis = GIS()

In [24]:
pd.read_csv('./data/results_pct_20241105.txt',sep='\t').head()

,County,Election Date,Precinct,Contest Group ID,Contest Type,Contest Name,Choice,Choice Party,Vote For,Election Day,Early Voting,Absentee by Mail,Provisional,Total Votes,Real Precinct
0,BURKE,11/05/2024,0072,1405,S,CONSTITUTIONAL AMENDMENT - CITIZENS-ONLY VOTING,For,,1,481,1665,66,11,2223,Y
1,CABARRUS,11/05/2024,01-02,1378,S,NC TREASURER,Wesley Harris,DEM,1,184,0,0,0,184,Y
2,CABARRUS,11/05/2024,01-02,1386,S,US HOUSE OF REPRESENTATIVES DISTRICT 08,Mark Harris,REP,1,460,0,0,0,460,Y
3,CABARRUS,11/05/2024,01-02,1393,S,US PRESIDENT,Cornel West,JFA,1,0,0,0,0,0,Y
4,CABARRUS,11/05/2024,01-04,1010,S,NC COURT OF APPEALS JUDGE SEAT 12,Tom Murry,REP,1,488,0,0,0,488,Y


In [29]:
df = pd.read_csv('./data/results_pct_20241105.txt',sep='\t')

#Filter for the presidential race
df = (df
      .loc[
          (df['Contest Name'] == 'US PRESIDENT')  
          #&(df['County'].isin(['DURHAM','WAKE','ORANGE']))
      ]
      .groupby(['County','Precinct','Choice Party'])
      .agg({'Total Votes':'sum'})
      .reset_index()
      .pivot_table(values='Total Votes',columns='Choice Party',index=['County','Precinct'])
      .reset_index()
     )
df['PctDEM'] = df['DEM'] / (df['DEM']+df['REP']) * 100
df.head()

Choice Party,County,Precinct,CST,DEM,GRE,JFA,LIB,REP,PctDEM
0,ALAMANCE,01,1,587,7,2,6,2431,19.449967
1,ALAMANCE,02,2,632,9,0,17,2558,19.811912
2,ALAMANCE,035,2,1642,10,8,9,1980,45.334070
3,ALAMANCE,03C,2,872,21,7,9,1124,43.687375
4,ALAMANCE,03N,3,1433,7,3,21,1150,55.478126


In [10]:
lyr_precincts = FeatureLayer('https://services.nconemap.gov/secure/rest/services/NC1Map_Boundaries/MapServer/0')
sdf_precincts = GeoAccessor.from_layer(lyr_precincts)
sdf_precincts.head()

,objectid,id,prec_id,enr_desc,county_nam,of_prec_id,county_id,shape_leng,st_area(shape),st_perimeter(shape),SHAPE
0,1,1,0003,ALBEMARLE NUMBER 3,STANLY,,84,40731.350779,43231281.474898,40731.350779,"{""rings"": [[[501353.6771267742, 178235.4578594..."
1,2,2,0003,DREXEL 03,BURKE,,12,49918.849039,108319906.647547,49918.84904,"{""rings"": [[[372074.70772677363, 221553.271259..."
2,3,22,0019,LINVILLE 01,BURKE,,12,160866.952809,871805502.088099,160866.952809,"{""rings"": [[[348092.06152677216, 224114.752159..."
3,4,23,0019,RIDENHOUR,STANLY,,84,125698.778851,899272261.409063,125698.778851,"{""rings"": [[[484800.94602677185, 182206.711059..."
4,5,71,007,007,MECKLENBURG,,60,17915.436972,18523838.740518,17915.436972,"{""rings"": [[[446820.3248267737, 159981.3444594..."


In [31]:
#Join voting data
sdf_precincts = lyr_precincts.query("county_nam IN ('WAKE','DURHAM','ORANGE')").sdf
sdf_precincts.head(2)

,objectid,id,prec_id,enr_desc,county_nam,of_prec_id,county_id,shape_leng,st_area(shape),st_perimeter(shape),SHAPE
0,26,495,07,SCHOOL OF THE ARTS,DURHAM,,32,23238.895291,18598541.386539,23238.895291,"{""rings"": [[[617530.541726774, 249263.70725940..."
1,39,105,01-05,01-05,WAKE,,92,14500.10594,13259014.500534,14500.10594,"{""rings"": [[[641594.0037267727, 227539.2814594..."


In [32]:
sdf_precincts_join = pd.merge(
    left=sdf_precincts,
    left_on = ['county_nam','prec_id'],
    right=df,
    right_on= ['County','Precinct'],
    how='left'
)
sdf_precincts_join.head()

,objectid,id,prec_id,enr_desc,county_nam,of_prec_id,county_id,shape_leng,st_area(shape),st_perimeter(shape),SHAPE,County,Precinct,CST,DEM,GRE,JFA,LIB,REP,PctDEM
0,26,495,07,SCHOOL OF THE ARTS,DURHAM,,32,23238.895291,18598541.386539,23238.895291,"{""rings"": [[[617530.541726774, 249263.70725940...",DURHAM,07,1.0,213.0,2.0,1.0,3.0,42.0,83.529412
1,39,105,01-05,01-05,WAKE,,92,14500.10594,13259014.500534,14500.10594,"{""rings"": [[[641594.0037267727, 227539.2814594...",WAKE,01-05,0.0,231.0,3.0,0.0,2.0,178.0,56.479218
2,165,212,02-01,02-01,WAKE,,92,43794.085684,105610338.848128,43794.085684,"{""rings"": [[[639662.6169267729, 238797.6794594...",WAKE,02-01,0.0,311.0,6.0,2.0,15.0,326.0,48.822606
3,53,110,01-09,01-09,WAKE,,92,12038.234692,8161908.762607,12038.234692,"{""rings"": [[[642164.7420267722, 227423.6200594...",WAKE,01-09,0.0,288.0,2.0,0.0,4.0,86.0,77.005348
4,63,727,12,MONUMENT OF FAITH CHURCH,DURHAM,,32,13937.78064,8738685.835032,13937.78064,"{""rings"": [[[619302.8264267727, 247152.5590594...",DURHAM,12,0.0,109.0,2.0,3.0,1.0,9.0,92.372881


https://developers.arcgis.com/python/latest/guide/visualizing-data-with-the-spatially-enabled-dataframe/  
https://developers.arcgis.com/python-2-3/api-reference/arcgis.features.toc.html#arcgis.features.GeoAccessor.plot

In [57]:
map1 = gis.map('Durham County, NC')

sdf_precincts_join.spatial.plot(
    map_widget=map1,
    palette='viridis',
    renderer_type='c',
    col='PctDEM',
    alpha=0.1,
    #Class break render options
    method='esriClassifyEqualInterval',
    class_count=10,
    line_width=0.1
)

map1

MapView(layout=Layout(height='400px', width='100%'))

In [45]:
map1 = gis.map('North Carolina')